In [1]:
import pandas as pd
import numpy as np

In [6]:
gdp = pd.read_csv("real_gdp_by_county.csv", encoding="latin1", skipfooter=4)

/var/folders/g0/1_gj42z94q93wf4k_ck_hqd40000gn/T/ipykernel_5257/1149125363.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  gdp = pd.read_csv("real_gdp_by_county.csv", encoding="latin1", skipfooter=4)


In [7]:
#define a list of rustbelt states
rb_codes = ["IL", "IN", "MI", "NY", "OH", "PA", "WV", "WI"]

#extacting state names from gdp df
state_codes = gdp["GeoName"].str.extract(r",\s*([A-Z]{2})")[0]

#creating a rustbelt state filter
rb_filter = state_codes.isin(rb_codes)

#gdp_rb.head()
rust_belt_only = True
if rust_belt_only:
    gdp_rb = gdp[rb_filter].copy()
else:
    gdp_rb = gdp[~gdp["GeoName"].str.contains("United States", na=False)].copy()


In [17]:
#extract year columns
year_cols = [c for c in gdp_rb.columns if c.isdigit()]

# wide to long
gdp_rb_long = gdp_rb.melt(
    id_vars=[c for c in gdp_rb.columns if c not in year_cols], 
    value_vars=year_cols,
    var_name="year",
    value_name="gdp_value"   
)

gdp_rb_long["year"] = gdp_rb_long["year"].astype(int)

gdp_rb_long = gdp_rb_long.drop(columns=["Region", "TableName", "LineCode"], errors="ignore")

gdp_rb_long["gdp_value"] = pd.to_numeric(gdp_rb_long["gdp_value"], errors="coerce")

gdp_rb_long = gdp_rb_long[gdp_rb_long["gdp_value"].notna()].copy()


In [18]:
gdp_components = (
    gdp_rb_long
    .pivot_table(
        index=["GeoFIPS", "GeoName", "year"],   
        columns="Description",                  
        values="gdp_value",
        aggfunc="first"                       
    )
    .reset_index()
)
gdp_components.columns = (
    gdp_components.columns
    .to_series()
    .astype(str)   
    .str.strip()   
    .values
)

In [19]:
gdp_components["GeoFIPS"] = (
    gdp_components["GeoFIPS"]
        .astype(str)          
        .str.strip()          
        .str.replace('"', '', regex=False)
        .astype("string")
        .astype("Int64")
)


In [20]:
save_copy = False
if save_copy == True: 
    gdp_components.to_csv("gdp_components_rust_belt.csv", index=False)

In [3]:
def clean_bea_data(bea_data, rb_codes, rust_belt_only=True):
    state_codes = bea_data["GeoName"].str.extract(r",\s*([A-Z]{2})")[0]
    rb_filter = state_codes.isin(rb_codes)
    if rust_belt_only:
        bea_rb = bea_data[rb_filter].copy()
    else:
        bea_rb = bea_data[~bea_data["GeoName"].str.contains("United States", na=False)].copy()
    year_cols = [c for c in bea_rb.columns if c.isdigit()]
    bea_rb_long = bea_rb.melt(
        id_vars=[c for c in bea_rb.columns if c not in year_cols], 
        value_vars=year_cols,
        var_name="year",
        value_name="value"   
    )
    bea_rb_long["year"] = bea_rb_long["year"].astype(int)
    bea_rb_long = bea_rb_long.drop(columns=["Region", "TableName", "LineCode"], errors="ignore")
    bea_rb_long["value"] = pd.to_numeric(bea_rb_long["value"], errors="coerce")
    bea_rb_long = bea_rb_long[bea_rb_long["value"].notna()].copy()
    components = (
        bea_rb_long
        .pivot_table(
            index=["GeoFIPS", "GeoName", "year"],   
            columns="Description",                  
            values="value",
            aggfunc="first"                       
        )
        .reset_index()
    )
    components.columns = (
        components.columns
        .to_series()
        .astype(str)   
        .str.strip()   
        .values
    )
    components["GeoFIPS"] = (
        components["GeoFIPS"]
            .astype(str)          
            .str.strip()          
            .str.replace('"', '', regex=False)
            .astype("string")
            .astype("Int64")
    )
    return components


In [25]:
income = pd.read_csv("income_county_industry.csv", encoding="latin1", skipfooter=4)

/var/folders/g0/1_gj42z94q93wf4k_ck_hqd40000gn/T/ipykernel_36635/2769428500.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  income = pd.read_csv("income_county_industry.csv", encoding="latin1", skipfooter=4)


In [29]:
income_cleaned = clean_bea_data(income, rb_codes, rust_belt_only=True)

In [30]:
save_copy = True
if save_copy: 
    income_cleaned.to_csv("income_cleaned.csv", index=False)

In [8]:
quant_index = pd.read_csv("quantity_index.csv", encoding="latin1", skipfooter=4)
quant_index_cleaned = clean_bea_data(quant_index, rb_codes, rust_belt_only=True)

/var/folders/g0/1_gj42z94q93wf4k_ck_hqd40000gn/T/ipykernel_5257/2960907463.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  quant_index = pd.read_csv("quantity_index.csv", encoding="latin1", skipfooter=4)


In [9]:
save_copy = True
if save_copy: 
    quant_index_cleaned.to_csv("quant_index_cleaned.csv", index=False)